In [1]:
from modal import Image, Secret, Stub, web_endpoint, enter, method
# from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from llm import get_anthropic_llm, get_openai_llm
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
import os
from typing import Dict
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
import modal
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from langchain_core.output_parsers import StrOutputParser
from pymongo import MongoClient
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import UnstructuredEPubLoader

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
%pip install pymongo langchain_nomic langchain_mongodb unstructured pypandoc

In [3]:
embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5")

# initialize MongoDB python client
client = MongoClient("mongodb+srv://sugamxp:<password>@cluster0.wsfteju.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")

DB_NAME = "mongohack"
COLLECTION_NAME = "judges"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "judges_index"

atlas_collection = client[DB_NAME][COLLECTION_NAME]

In [9]:
loader = UnstructuredEPubLoader("cuban.epub")
data = loader.load()
# Split PDF into documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
docs = text_splitter.split_documents(data)
# Print the first document
print(docs[101])

page_content='Investors don’t care about your dreams and goals. They love that you\nhave them. They love that you are motivated by them. Investors care\nabout how they are going to get their money back and then some. Family\ncares about your dreams. Investors care about money. There is a reason\nwhy venture capitalists are often referred to as Vulture Capitalists.\nThe minute you slide off course from the promises you made to get the\nmoney, your dreams fall in jeopardy. You will find yourself making\npromises to keep investors at bay. You will find yourself avoiding your\ninvestors. Then you will find yourself on the outside looking in. The\nreality of taking money from non-family members is that they are doing\nit for only one reason, to make more money. If you can’t deliver on that\npromise, you are out. You will be removed from the company you started.\nYou w star backill find someone else running your dream company. If this\nsounds like a scene out of The Sopranos or' metadata={'s

In [10]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = embeddings,
    collection = atlas_collection,
    index_name = ATLAS_VECTOR_SEARCH_INDEX_NAME
)

retriever = vector_search.as_retriever(
   search_type = "similarity",
   search_kwargs = {"k": 10, "score_threshold": 0.75}
)